# ELM

## Importing Libraries

In [11]:
import torch
import time
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# Dateset Class and DataLoader

In [12]:
class ELM_Dataset(Dataset):
    def __init__(self, features, labels):
        super(ELM_Dataset, self).__init__()

        self.features = torch.Tensor(features, dtype = torch.float32)
        self.labels = torch.Tensor(labels, dtype = torch.float32)

    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]

        return feature, label

## ELM Class

In [13]:
class ELM(nn.Module):
    def __init__(self, n_hidden_neurons, device = 'cuda', chunk_size = 100):
        super(ELM, self).__init__()

        self.n_hidden_neurons = n_hidden_neurons
        self.device = device
        self.chunk_size = chunk_size
        self.input_weights = None
        self.biases = None
        self.output_weights = None

    def fit(self, X, y, regularization = 1e-8):
        
        X = X.to(self.device)
        y = y.to(self.device).view(-1, 1)
        
        n_samples, n_features = X.shape

        self.input_weights = torch.randn(n_features, self.n_hidden_neurons, device = self.device, dtype = torch.float32)
        self.biases = torch.randn(1, self.n_hidden_neurons, device = self.device, dtype = torch.float32)

        # Check if we need micro-batching based on memory requirements
        if torch.cuda.is_available():
            available_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
            required_mem = n_samples * self.n_hidden_neurons * 4 / 1e9  # 4 bytes per float32
            
            if required_mem > available_mem * 0.6:  # Use micro-batching if H matrix > 60% of GPU memory
                print(f"Large matrix detected ({required_mem:.1f} GB). Using micro-batching...")
                return self._fit_micro_batched(X, y, regularization)
        
        # Standard ELM computation
        try:
            H = torch.sigmoid(X @ self.input_weights + self.biases)
            HTH = H.T @ H
            HTy = H.T @ y
        except torch.cuda.OutOfMemoryError:
            print("GPU out of memory. Switching to micro-batching...")
            torch.cuda.empty_cache()
            return self._fit_micro_batched(X, y, regularization)

        I = torch.eye(self.n_hidden_neurons, device=self.device)
        self.output_weights = torch.linalg.solve(HTH + regularization * I, HTy)

    def _fit_micro_batched(self, X, y, regularization):
        """Micro-batching to handle large matrices"""
        n_samples = X.shape[0]
        
        # Initialize accumulator matrices
        HTH = torch.zeros(self.n_hidden_neurons, self.n_hidden_neurons, device=self.device, dtype=torch.float32)
        HTy = torch.zeros(self.n_hidden_neurons, 1, device=self.device, dtype=torch.float32)
        
        # Process in small chunks
        for i in range(0, n_samples, self.chunk_size):
            end_idx = min(i + self.chunk_size, n_samples)
            X_chunk = X[i:end_idx]
            y_chunk = y[i:end_idx]
            
            # Compute hidden layer for chunk
            H_chunk = torch.sigmoid(X_chunk @ self.input_weights + self.biases)
            
            # Accumulate HTH and HTy
            HTH += H_chunk.T @ H_chunk
            HTy += H_chunk.T @ y_chunk
            
            # Clear chunk from memory
            del H_chunk
            
        # Solve for output weights using accumulated matrices
        I = torch.eye(self.n_hidden_neurons, device=self.device)
        self.output_weights = torch.linalg.solve(HTH + regularization * I, HTy)

    def predict(self, X):

        X = X.to(self.device)
        n_samples = X.shape[0]
        
        # Use micro-batching for prediction if needed
        if n_samples > self.chunk_size * 2:  # Use chunking for large prediction sets
            predictions_list = []
            
            for i in range(0, n_samples, self.chunk_size):
                end_idx = min(i + self.chunk_size, n_samples)
                X_chunk = X[i:end_idx]
                
                H_chunk = torch.sigmoid(X_chunk @ self.input_weights + self.biases)
                pred_chunk = H_chunk @ self.output_weights
                predictions_list.append((pred_chunk > 0.5).int().flatten())
                
                del H_chunk, pred_chunk
            
            return torch.cat(predictions_list, dim=0)
        
        # Standard prediction
        H = torch.sigmoid(X @ self.input_weights + self.biases)
        predictions = H @ self.output_weights

        return (predictions > 0.5).int().flatten()

In [14]:
# print("Loading and preparing MEFAR_MID.csv data...")
# df = pd.read_csv('../Datasets/MEFAR_MID.csv')

# # Handle missing values
# if df.isnull().sum().sum() > 0:
#     df.fillna(df.mean(), inplace=True)

# X = df.iloc[:, :-1].values
# y = df.iloc[:, -1].values

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# X_train_pt = torch.tensor(X_train_scaled, dtype=torch.float32)
# y_train_pt = torch.tensor(y_train, dtype=torch.float32)
# X_test_pt = torch.tensor(X_test_scaled, dtype=torch.float32)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # --- 2. Set up the Tuning Loop ---
# # Define the range of hidden neuron values to test
# neuron_counts = [3000, 3500, 4000, 5000, 6000, 7000, 8000, 10000]
# accuracies = []

# print(f"\nStarting hyperparameter tuning on {device}...")

# for n_neurons in neuron_counts:
#     # Train the model with the current number of neurons
#     model = ELM(n_hidden_neurons=n_neurons)
#     model.fit(X_train_pt, y_train_pt)
    
#     # Evaluate the model
#     predictions_pt = model.predict(X_test_pt)
#     predictions_np = predictions_pt.cpu().numpy()
    
#     acc = accuracy_score(y_test, predictions_np)
#     accuracies.append(acc)
    
#     print(f"Neurons = {n_neurons}, Accuracy = {acc * 100:.2f}%")

# # --- 3. Find and Print the Best Result ---
# best_accuracy = max(accuracies)
# best_n_neurons = neuron_counts[accuracies.index(best_accuracy)]

# print(f"\nTuning Complete!")
# print(f"Best Accuracy: {best_accuracy * 100:.2f}% achieved with {best_n_neurons} hidden neurons.")

# # --- 4. Plot the Results ---
# plt.figure(figsize=(10, 6))
# plt.plot(neuron_counts, accuracies, marker='o', linestyle='-')
# plt.title('ELM Performance vs. Number of Hidden Neurons')
# plt.xlabel('Number of Hidden Neurons')
# plt.ylabel('Accuracy')
# plt.xticks(neuron_counts)
# plt.grid(True)
# plt.show()

## Initialization and Model Training

In [ ]:
HIDDEN_NEURONS = 20000
DATASET_PATH = '../Datasets'

dataset_list = os.listdir(path = DATASET_PATH)

for filename in dataset_list:

    #Checking for GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name()}")
        print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


    #Processing each Dataset
    print(f'Processing {filename}')

    file_path = f'../Datasets/{filename}'
    df = pd.read_csv(file_path)

    #Checking and Handling Missing Values
    if df.isnull().sum().sum() > 0:
        print("Found missing values. Filling with column mean.")
        df.fillna(df.mean(), inplace=True)

    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_pt = torch.tensor(X_train_scaled, dtype = torch.float32)
    y_train_pt = torch.tensor(y_train, dtype = torch.float32)
    X_test_pt = torch.tensor(X_test_scaled, dtype = torch.float32)

    model = ELM(n_hidden_neurons = HIDDEN_NEURONS, device = device)

    # Checking Training Time
    start_time = time.time()
    model.fit(X_train_pt, y_train_pt)
    training_time = time.time() - start_time

    predictions = model.predict(X_test_pt)
    predictions_np = predictions.cpu().numpy()

    accuracy = accuracy_score(y_test, predictions_np)

    report = classification_report(y_test, predictions_np, zero_division = 0)

    print(f'Accuracy for {filename}: {accuracy * 100:.2f}%\n')
    print('Classification Report')
    print(report)
    print('-------------------------------------------')




Using device: cuda
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Available GPU memory: 8.6 GB
Processing MEFAR_DOWN.csv
Found missing values. Filling with column mean.
Accuracy for MEFAR_DOWN.csv: 51.18%

Classification Report
              precision    recall  f1-score   support

         0.0       0.50      0.52      0.51      2673
         1.0       0.53      0.51      0.52      2841

    accuracy                           0.51      5514
   macro avg       0.51      0.51      0.51      5514
weighted avg       0.51      0.51      0.51      5514

-------------------------------------------
Using device: cuda
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Available GPU memory: 8.6 GB
Processing MEFAR_MID.csv
Large matrix detected (59.1 GB). Using micro-batching...
Large matrix detected (59.1 GB). Using micro-batching...
Accuracy for MEFAR_MID.csv: 94.57%

Classification Report
              precision    recall  f1-score   support

         0.0       0.94      0.95      0.95     92491
         1.0  

In [17]:
def tune_hidden_neurons(X_train, y_train, X_test, y_test, neuron_range=None, device='cuda'):
    """
    Tune the number of hidden neurons for base ELM
    """

    df = pd.read_csv('../Datasets/MEFAR_MID.csv')

    # Handle missing values
    if df.isnull().sum().sum() > 0:
        df.fillna(df.mean(), inplace=True)

    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_pt = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_pt = torch.tensor(y_train, dtype=torch.float32)
    X_test_pt = torch.tensor(X_test_scaled, dtype=torch.float32)

    if neuron_range is None:
        neuron_range = [10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]
    
    X_train_pt = X_train_pt.to(device)
    y_train_pt = y_train_pt.to(device)
    X_test_pt = X_test_pt.to(device)
    
    results = []
    
    print("Tuning number of hidden neurons...")
    for n_neurons in neuron_range:
        model = ELM(n_hidden_neurons=n_neurons, device=device, chunk_size=100)
        
        start_time = time.time()
        model.fit(X_train_pt, y_train_pt)
        training_time = time.time() - start_time
        
        predictions = model.predict(X_test_pt)
        accuracy = accuracy_score(y_test, predictions.cpu().numpy())
        
        results.append({
            'neurons': n_neurons,
            'accuracy': accuracy,
            'training_time': training_time
        })
        
        print(f"Neurons: {n_neurons:5d} | Accuracy: {accuracy*100:6.2f}% | Time: {training_time:.3f}s")
        
        # Clear cache after each test
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Find best configuration
    best_result = max(results, key=lambda x: x['accuracy'])
    print(f"\nBest configuration:")
    print(f"Neurons: {best_result['neurons']}")
    print(f"Accuracy: {best_result['accuracy']*100:.2f}%")
    print(f"Training time: {best_result['training_time']:.3f}s")
    
    return results

In [19]:
tune_hidden_neurons(X_train, y_train, X_test, y_test, neuron_range=None, device='cuda')

Tuning number of hidden neurons...
Large matrix detected (29.5 GB). Using micro-batching...
Neurons: 10000 | Accuracy:  88.54% | Time: 52.458s
Large matrix detected (32.5 GB). Using micro-batching...
Neurons: 11000 | Accuracy:  89.33% | Time: 63.200s
Large matrix detected (35.5 GB). Using micro-batching...
Neurons: 12000 | Accuracy:  89.79% | Time: 75.134s
Large matrix detected (38.4 GB). Using micro-batching...
Neurons: 13000 | Accuracy:  90.66% | Time: 88.286s
Large matrix detected (41.4 GB). Using micro-batching...
Neurons: 14000 | Accuracy:  91.41% | Time: 102.376s
Large matrix detected (44.3 GB). Using micro-batching...
Neurons: 15000 | Accuracy:  91.79% | Time: 117.471s
Large matrix detected (47.3 GB). Using micro-batching...
Neurons: 16000 | Accuracy:  92.52% | Time: 133.515s
Large matrix detected (50.2 GB). Using micro-batching...
Neurons: 17000 | Accuracy:  93.07% | Time: 150.764s
Large matrix detected (53.2 GB). Using micro-batching...
Neurons: 18000 | Accuracy:  93.64% | Tim

[{'neurons': 10000,
  'accuracy': 0.885421856384707,
  'training_time': 52.45798134803772},
 {'neurons': 11000,
  'accuracy': 0.8932578793458248,
  'training_time': 63.19957518577576},
 {'neurons': 12000,
  'accuracy': 0.8979205025452183,
  'training_time': 75.13354444503784},
 {'neurons': 13000,
  'accuracy': 0.9065579984836998,
  'training_time': 88.28577899932861},
 {'neurons': 14000,
  'accuracy': 0.9141124228311491,
  'training_time': 102.37623238563538},
 {'neurons': 15000,
  'accuracy': 0.9178598505361204,
  'training_time': 117.4709951877594},
 {'neurons': 16000,
  'accuracy': 0.9251976605653633,
  'training_time': 133.5154423713684},
 {'neurons': 17000,
  'accuracy': 0.9306671721000758,
  'training_time': 150.76416730880737},
 {'neurons': 18000,
  'accuracy': 0.9364236976064118,
  'training_time': 169.10250687599182},
 {'neurons': 19000,
  'accuracy': 0.9408805372035092,
  'training_time': 189.15259957313538},
 {'neurons': 20000,
  'accuracy': 0.9463933716018629,
  'training_t

In [ ]:
best_result = max(results, key=lambda x: x['accuracy'])
print(f"\nBest configuration:")
print(f"Neurons: {best_result['neurons']}")
print(f"Accuracy: {best_result['accuracy']*100:.2f}%")
print(f"Training time: {best_result['training_time']:.3f}s")